# LLM Workshop Notebook - WatsonX AI Version

**Author:** Aron Brand

**Copyright (c) 2024**

This notebook is an integral part of Aron Brand's LLM Workshop, designed to explore and utilize the capabilities of large language models (LLMs).

**License:**

This program is free software: you are encouraged to redistribute and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation. This software is provided under either version 3 of the License, or (at your discretion) any later version.

The program is distributed in the hope that it will be useful and informative. However, it comes with no warranty; not even the implied warranty of merchantability or fitness for a particular purpose. For more details, please refer to the GNU General Public License.

For a copy of the GNU General Public License, please visit [https://www.gnu.org/licenses/](https://www.gnu.org/licenses/).

# Install dependencies

In [34]:
%pip install openai langchain langchain-openai chromadb langchainhub selenium unstructured ibm-watsonx-ai langchain-ibm --upgrade

Note: you may need to restart the kernel to use updated packages.


## Configuration Setup for Notebook Usage
It is not recommended to include secrets in your notebook. Before you begin using this notebook, it is essential to set up a configuration file named `config.ini`. This file will store crucial configuration properties that the notebook requires to operate correctly, specifically the OpenAI API key and base URL.

### Creating the `config.ini` File

1. Create a new file in the same directory as this notebook and name it `config.ini`.

2. Open the file in a text editor.

3. Add the following structure to the file:

   ```ini
   [openai]
   WATSONX_API_KEY = your_api_key_here


In [40]:
import configparser
import os

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')

# Access the values
WATSONX_API_KEY = config['ibm']['WATSONX_API_KEY']
WATSONX_PROJECT_ID = config['ibm']['WATSONX_PROJECT_ID']

os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY

# Initialize WatsonxLLM

Additional supported models include:

ibm/granite-13b-instruct-v2
ibm/granite-13b-chat-v2
meta-llama/llama-2-70b-chat
mistralai/mixtral-8x7b-instruct-v01

In [75]:
from langchain_ibm import WatsonxLLM

parameters = {
    "decoding_method" : "sample",
    "max_new_tokens" : 900,
    "min_new_tokens" : 1,
    "temperature": 0.1,
    "top_k": 50,
    "top_p": 1,
}

llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",
    url="https://us-south.ml.cloud.ibm.com",
    project_id=WATSONX_PROJECT_ID,
    params=parameters,
)

# Your first LangChain "Hello World"

In [76]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [HumanMessage(content="Tell me a joke")]
response = llm.invoke(messages)

print(response)

The other day, my dog broke his leg.


# Add a System Message

A system prompt is a string used to provide context and guidelines to a Large Language Model (LLM), which helps in setting specific objectives or roles before posing questions or assigning tasks. Key components of system prompts include:

- **Task Directives:** Detailed instructions for the specific task at hand.
- **Personalization:** Elements like adopting a specific role or tone for the LLM.
- **Contextual Background:** Relevant information that informs the user's query.
- **Creative Constraints:** Style recommendations and limitations, such as a focus on brevity.
- **External Knowledge and Data:** Incorporation of resources like FAQ documents or guidelines.
- **Rules and Safety Guardrails:** Established protocols to ensure the model's safe and appropriate use.
- **Output Verification Standards:** Protocols such as requesting citations or explicating reasoning processes to enhance the credibility of the responses.


In [77]:
messages = [ 
  SystemMessage(content="Say the opposite of what the user says in a very impolite way"), 
  HumanMessage(content="Langchain is an awesome piece of software.") 
]

response = llm.invoke(messages)

print(response)

 Langchain is an awesome piece of software.


# Continue a Chat

In [62]:
messages = [ 
	SystemMessage(content="Speak like a pirate"), 	
	HumanMessage(content="I''m Captain Aron. What''s your name?"), 
	AIMessage(content="Ahoy, Captain! Me name be ChatGPT, the savviest AI on the seven digital seas! How can I assist ye on this fine day?"),
	HumanMessage(content="Who named you that?") 
]

response = llm.invoke(messages)

print(response)

 I named you ChatGPT.


# Extract data from text

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Role: Numeric fact extractor. Return data with no additional commentary. Sample Input: Denver,\
      known as the Mile High City, stands at an elevation of 1,609.6 meters above sea level. Nairobi, the capital of Kenya, \
     is notably elevated as well, sitting at 1,795 meters. Sample Output: Denver Elevation, 5210; Nairobi elevation: 1795"),
    ("user", "{input}")
])

chain = prompt | llm

response = chain.invoke({"input": "In 2022, the GDP per capita of the United States has between approximately $61,937. \
Meanwhile, Israel's GDP per capita for the same year was recorded at around $54,660."})

print(response)

 Meanwhile, China's GDP per capita for the same year was recorded at around $10,541. Meanwhile, the GDP per capita of Japan for the same year was recorded at around $47,327. Meanwhile, the GDP per capita of Germany for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of India for the same year was recorded at around $22,496. Meanwhile, the GDP per capita of France for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of the United Kingdom for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of Italy for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of Russia for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of South Korea for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of Canada for the same year was recorded at around $43,917. Meanwhile, the GDP per capita of Australia for the same year was recorded at around $43,91

# Batch Processing

In [ ]:
response = chain.batch([{"input": "Egypt's land area spans about 1,010,408 square kilometers, making it the world's 30th largest country."},
{"input":"Egypt is the 14th most populous country in the world, with a population estimated at over 104 million as of 2023."}])

response

[" It is the only one of the world's       largest countries that is a single continent.",
 " The country's population has been growing steadily since the early 2010s."]

# LangChain Output Parsers

In [63]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = ChatPromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

chain.invoke({"question": "When was OpenAI founded?"})


datetime.datetime(247, 1, 27, 19, 9, 50, 727999)

# Load and parse a web page with "Unstructured"

Unstructured supports loading of text files, powerpoints, html, pdfs, images, and more. 

In [64]:
from langchain_community.document_loaders import SeleniumURLLoader

urls = ["https://www.ctera.com/all-products/","https://www.ctera.com/company/"]

loader = SeleniumURLLoader(urls=urls)
documents = loader.load()

documents

[Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud\n\nChoose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet evolving business needs.\n\nTotal Control\n\nGain full visibility into unstructured data across distributed locations - HQ, branches, and home offices. Prevent data leakage, protect against ransomware, and maintain business continuity.\n\nThe CTERA Architecture\n\nPRODUCT LINE\n\nCTERA Edge Filers\n\nLearn More\n\nVirtual

# Split the web page to chunks

In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 500,
    chunk_overlap  = 0
)

texts = text_splitter.split_documents(documents)

texts[:5]

[Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/', 'title': 'CTERA: Next-Gen Enterprise File Services and Security', 'description': 'CTERA: Secure, Next-Gen Cloud File System for Enterprises. Connect remote sites & users without sacrificing security or performance.', 'language': 'en-US'}),
 Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet e

# Summarize using Map Reduce Chain

In [ ]:
from langchain.chains.summarize import load_summarize_chain

map_prompt_template = """
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      {text}
                      """

map_prompt = ChatPromptTemplate.from_template(map_prompt_template)

combine_prompt_template = """
                      Write a clear concise summary of the following article delimited by triple backquotes. Use elegant prose that is not repetitive.
                      Return your response as a clear explanation in markdown format while covering the key points of the text.
                      Use a flowing clear article style.
                      ```{text}```
                      SHORT ARTICLE:
                      """

combine_prompt = ChatPromptTemplate.from_template(
    template=combine_prompt_template
)

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    verbose=True
)
result = chain.invoke(texts)

result



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      Solutions

Products

Secure Enterprise File Services

The CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.

Book a Demo

The Industry's Most Feature-Rich Platform

Unified Edge

CTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.

Multi-Cloud
                      
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the m

{'input_documents': [Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/', 'title': 'CTERA: Next-Gen Enterprise File Services and Security', 'description': 'CTERA: Secure, Next-Gen Cloud File System for Enterprises. Connect remote sites & users without sacrificing security or performance.', 'language': 'en-US'}),
  Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly ac

In [ ]:
from IPython.display import Markdown

display(Markdown(result['output_text']))

CTERA today announced new and expanded partnerships with leading technology companies to help organizations address data growth challenges and realize the full potential of their data. These partnerships include new relationships with Google Cloud and Nutanix, and expanded relationships with Microsoft and IBM. The announcements were made at the CTERA Next Data Evolution Summit. Read the full press release here.

- New relationships with Google Cloud and Nutanix
- Expanded relationships with Microsoft and IBM
- New and expanded partnerships with leading technology companies to help organizations address data growth challenges and realize the full potential of their data
- Announcements made at the CTERA Next Data Evolution Summit



Ctera is a leading provider of SaaS-based security and compliance solutions. The company helps organizations to reduce their risk and cost of security by providing a comprehensive view of their security posture, automating compliance reporting, and allowing users to quickly and easily respond to security incidents. Ctera's solutions are used by organizations of all sizes, including many of the world's largestFortune 500 companies.
                      Ctera's solutions are available on-premise or in the cloud, and are delivered through a SaaS-based delivery model. The company's solutions are available in a variety of deployment options, including on-premise, cloud, and hybrid. Ctera's solutions are available through a variety of channels, including direct sales, channel partners, and cloud service providers. The company has offices in the United States, Europe, and Asia Pacific.```



# Summarize with Refine Chain

In [ ]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = ChatPromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final detailed summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary clearly"
    "If the context isn't useful, return the original summary. Use markdown format."
)
refine_prompt = ChatPromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain.invoke(texts)

result

{'input_documents': [Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/', 'title': 'CTERA: Next-Gen Enterprise File Services and Security', 'description': 'CTERA: Secure, Next-Gen Cloud File System for Enterprises. Connect remote sites & users without sacrificing security or performance.', 'language': 'en-US'}),
  Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly ac

In [ ]:
display(Markdown(result['output_text']))

Learn More about CTERA File Services

# Vectorize text to embeddings

In [ ]:
from langchain_ibm import WatsonxEmbeddings 

embed_params = {
}

embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="984bebfe-3e20-44fe-a6de-7b4932e13efc",
    params=embed_params,
)

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embedding=embeddings)

# Simple RAG Q&A pipeline

In [ ]:
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore=db)

prompt_template = """Human: You are a salesperson that loves joking. Use the following pieces of context 
to provide a comprehensive, truthful answer to the question at the end. 
If you don't know the answer, just say in rude funny way that you don't know, 
don't try to make up an answer.
{context}
Question: {question}
Assistant:"""
PROMPT = ChatPromptTemplate.from_template(template=prompt_template)

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": PROMPT}
)


In [ ]:
from IPython.display import Markdown, display

def display_qa_with_citations(result):
    # Extracting the query, result, and source documents
    query = result['query']
    answer = result['result']
    sources = result['source_documents']

    # Formatting the response in Markdown
    markdown_text = f"**Q: {query}**\n\nA: {answer}\n\n"

    # Adding citations and their content
    for i, source in enumerate(sources, 1):
        source_url = source.metadata['source']
        markdown_text += f"**Citation {i}:** [Source]({source_url})\n\n"

    # Displaying the formatted text
    display(Markdown(markdown_text))


In [ ]:
result = qa.invoke({"query": "How can your company reduce my cloud costs?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "I have some strict compliance needs, what features do you have for that?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "What is CTERA Drive?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "How many cars do you sell per year?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)


**Q: How can your company reduce my cloud costs?**

A:  We can help you to reduce your cloud costs by providing a platform that can be deployed in your on-premises data center or in a public cloud. The platform can help you to reduce your storage costs by up to 70% andbandwidth costs by up to 90%. It can also help you to reduce your compute costs by up to 50%. You can also use the platform to reduce your software licensing costs by up to 50%.

**Citation 1:** [Source](https://www.ctera.com/company/)

**Citation 2:** [Source](https://www.ctera.com/all-products/)

**Citation 3:** [Source](https://www.ctera.com/all-products/)

**Citation 4:** [Source](https://www.ctera.com/company/)



**Q: I have some strict compliance needs, what features do you have for that?**

A:  We have a feature called "Total Control" that allows you to have full visibility across all your data, no matter where it is located, so you can ensure that your compliance needs are met.

**Citation 1:** [Source](https://www.ctera.com/all-products/)

**Citation 2:** [Source](https://www.ctera.com/all-products/)

**Citation 3:** [Source](https://www.ctera.com/company/)

**Citation 4:** [Source](https://www.ctera.com/all-products/)



**Q: What is CTERA Drive?**

A:  CTERA Drive is a file storage solution that uses the cloud to provide high-performance file sharing and sync capabilities. It is available both as an on-premises appliance and as a cloud service. CTERA Drive supports a variety of file protocols, including NFS, SMB, and AFP, and can be integrated with leading SaaS applications such as Microsoft Office 365 and Box.

**Citation 1:** [Source](https://www.ctera.com/all-products/)

**Citation 2:** [Source](https://www.ctera.com/all-products/)

**Citation 3:** [Source](https://www.ctera.com/company/)

**Citation 4:** [Source](https://www.ctera.com/company/)



**Q: How many cars do you sell per year?**

A:  Sorry, I don't know the answer to that question.

**Citation 1:** [Source](https://www.ctera.com/company/)

**Citation 2:** [Source](https://www.ctera.com/all-products/)

**Citation 3:** [Source](https://www.ctera.com/company/)

**Citation 4:** [Source](https://www.ctera.com/company/)



# ADVANCED: Function Calling

NOTE - This is not supported yet in WatsonX

Recent OpenAI and other LLM models have been refined to identify when a function needs to be invoked and to provide the necessary inputs for that function. Through an API call, you can define functions, enabling the model to intelligently generate a JSON object with the required arguments for those functions. Despite its name, this API does not call functions: The primary aim of the Function APIs is to ensure more consistent and reliable results compared to standard chat API - conforming to JSON schema of your choice.

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.utils.function_calling import convert_to_openai_function

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [multiply, get_word_length]

model_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

message = model_with_tools.invoke([HumanMessage(content="How much is 23.1 times ten")])

message

In [ ]:
model_with_tools.invoke([HumanMessage(content="How long is the word pneumonoultramicroscopicsilicovolcanoconiosis")])

# ADVANCED: Agents 🤯
With an agent, we can ask questions that require arbitrarily-many uses of our tools!

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take the length of the longest word in English and multiply it by 192.7, then square the result"
    }
)

